In [1]:
!pip install 'smolagents[litellm]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 54.7 MB/s eta 0:00:00


In [2]:
pip install smolagents requests beautifulsoup4 duckduckgo-search huggingface_hub diffusers transformers matplotlib

In [15]:
from smolagents import CodeAgent, DuckDuckGoSearchTool
from smolagents.models import LiteLLMModel
from google.colab import userdata

api_key = userdata.get("GEMINI_KEY")

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-lite-001",
    temperature=0.7,
    api_key=api_key
)

duck_search_tool = DuckDuckGoSearchTool()

retriever_agent = CodeAgent(
    tools=[duck_search_tool],
    model=model,
    name="retriever_agent",
    description="Fetches weather information for a given location."
)

search_agent = CodeAgent(
    tools=[duck_search_tool],
    model=model,
    name="search_agent",
    description="Performs searches to find city country information."
)

In [13]:
!pip install transformers sentencepiece

In [16]:
def extract_city(query):
    # A simple regex that extracts capitalized words as cities (improve this for real use)
    cities = re.findall(r"\b[A-Z][a-z]+\b", query)
    if cities:
        return cities[0]
    return None

# Country-language map (expand as needed or load from dataset)
country_language_map = {
    "Germany": "de",
    "United Kingdom": "en",
    "France": "fr",
    "Spain": "es",
    "Italy": "it",
    "Canada": "en",
    "United States": "en"
}


In [18]:
from transformers import MarianMTModel, MarianTokenizer
import re

In [19]:
def load_translator(src_lang: str, tgt_lang: str):
    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

def translate(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

In [20]:
def find_country(city):
    query = f"{city} country"
    result = search_agent.run(query)

    for country in country_language_map.keys():
        if country.lower() in result.lower():
            return country
    return None

In [21]:

def manager_agent(query):
    city = extract_city(query)
    if not city:
        city = "London"

    country = find_country(city)
    if not country:
        country = "United Kingdom"

    target_lang = country_language_map.get(country, "en")

    weather_info = retriever_agent.run(query)

    if target_lang != 'en':
        tokenizer, model = load_translator("en", target_lang)
        translated_text = translate(weather_info, tokenizer, model)
        return translated_text
    else:
        return weather_info


In [22]:
print(manager_agent("Berlin weather today"))

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ Berlin country                                                                                                  │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite-001 ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  country = web_search(query="Berlin country")                                                                     
  print(country)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Berlin - Wikipedia](https://en.wikipedia.org/wiki/Berlin)
Berlin is a global city of culture, politics, media and science, with a population of over 3.7 million. It is the 
capital and largest city of Germany, and one of the states of the country, surrounded by the state of Brandenburg.

[Berlin | History, Map, Population, Attractions, & Facts | Britannica](https://www.britannica.com/place/Berlin)
Berlin is situated about 112 miles (180 km) south of the Baltic Sea, 118 miles (190 km) north of the Czech-German 
border, 110 miles (177 km) east of the former inner-German border, and 55 miles (89 km) west of Poland.It lies in 
the wide glacial valley of the Spree River, which runs through the centre of the city.The mean elevation of Berlin 
is 115 feet (35 metres) above sea level.

[Berlin - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Berlin)
Berlin is the capital city of Germany and the largest city in the European Union. It has a rich history, a diverse 
population and many famous landmarks and attractions.

[Berlin - New World Encyclopedia](https://www.newworldencyclopedia.org/entry/Berlin)
Berlin is the capital city and one of the 16 states of the Federal Republic of Germany. It has a rich history, a 
diverse culture, and a vibrant economy, and is located in northeastern Germany near the border with Poland.

[Berlin - Profile of the City and German Federal State](https://www.nationsonline.org/oneworld/Germany/berlin.htm)
Learn about Berlin, the largest city and capital of Germany, located in the state of Brandenburg. Find information 
on Berlin's geography, history, culture, economy, tourism, education and more.

[Berlin - Students | Britannica Kids | Homework Help](https://kids.britannica.com/students/article/Berlin/273186)
Learn about Berlin, the capital and largest city of Germany, and its history, culture, and attractions. Find out 
how Berlin was divided by the Cold War and reunified in 1990.

[Where is Berlin | Location of Berlin in Germany Map - Maps of 
World](https://www.mapsofworld.com/where-is/berlin.html)
1.7K. Berlin, the vibrant capital of Germany, is like the eclectic spot where historic remnants meet contemporary 
creativity. With its iconic Brandenburg Gate, the scenic Spree River, and a mix of trendy art galleries, it's a 
city that embraces its tumultuous history while being a hotbed of innovation and diverse subcultures.

[Where is Berlin, Germany?](https://www.whereig.com/germany/berlin-location-map.html)
Berlin is the capital and largest city of Germany, located in the north-east region of the country. Learn about its
history, geography, climate, attractions, and more on this web page.

[History of Berlin - Wikipedia](https://en.wikipedia.org/wiki/History_of_Berlin)
History of Berlin; Margraviate of Brandenburg (1157-1806) Kingdom of Prussia (1701-1918) German Empire (1871-1918) 
Free State of Prussia (1918-1947) ... Berlin was the second biggest inland harbor in the country. All this 
infrastructure was needed to transport and feed the over 4 million Berliners. Before the 1929 crash, 450,000 people
...

[What is the Capital of Germany? - Mappr](https://www.mappr.co/capital-cities/germany/)
Berlin is the political and cultural heart of Germany, located in northeastern Germany near the border with Poland.
Learn about its complex and fascinating history, from its origins as a fishing settlement to its role as the 
capital of the reunified Germany.

Out: None

[Step 1: Duration 2.13 seconds| Input tokens: 2,209 | Output tokens: 53]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Das Wetter in Berlin liegt heute bei 17-25°C.


In [23]:
print(manager_agent("Paris weather today"))

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ Paris country                                                                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite-001 ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  country = web_search(query="Paris country")                                                                      
  print(country)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Paris | Definition, Map, Population, Facts, & History | Britannica](https://www.britannica.com/place/Paris)
Paris occupies a central position in the rich agricultural region known as the Paris Basin, and it constitutes one 
of eight départements of the Île-de-France administrative region. It is by far the country's most important centre 
of commerce and culture.Area city, 41 square miles (105 square km); metropolitan area, 890 square miles (2,300 
square km).

[Which country is Paris in? - Geographic FAQ Hub: Answers to ... - 
NCESC](https://www.ncesc.com/geographic-faq/which-country-is-paris-in/)
Which Country is Paris In? The answer is straightforward: Paris is in France. It's the capital and most populous 
city of the country, playing a vital role in its political, economic, and cultural landscape. Paris is situated on 
the Seine River in the north-central part of France, nestled within the Île-de-France region, often referred to …

[Geography of Paris - Wikipedia](https://en.wikipedia.org/wiki/Geography_of_Paris)
Learn about the location, area, climate, topography and geology of Paris, the capital of France and a major 
European city. Paris is located in northern central France, on the river Seine, and has a flat terrain with some 
hills and a rich history of meteorological observations.

[Paris - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Paris)
Paris is the capital and largest city of France, located on the Seine river. It has a rich history, culture, and 
attractions, such as the Eiffel Tower, Notre-Dame, and the Louvre museum.

[Where is Paris? - World In Paris](https://worldinparis.com/where-is-paris)
Learn about the geographical and administrative location of Paris, France, in Europe and the world. Find out 
interesting facts, practical information, and useful addresses for your trip to Paris.

[Which state is Paris? - StudyCountry.com](https://www.studycountry.com/wiki/which-state-is-paris)
Paris is the capital city of France. The city is a commune and department , and the heart of the 
12,012-square-kilometre (4,638-square-mile) Île-de-France ... France is a unitary semi-presidential republic with 
its capital in Paris, the country's largest city and main cultural and commercial centre; other major urban areas 
include Marseille ...

[Paris - capital city of France - The World Countries](https://theworldcountries.com/place/paris/)
Learn about the history, geography, culture, and attractions of Paris, the city and capital of France. Explore its 
landmarks, museums, parks, and rivers, and discover its role as a global centre of arts, education, and commerce.

[Where Is Paris Located in Which Country? - 
LuxuryTravelDiva](https://luxurytraveldiva.com/where-is-paris-located-in-which-country/)
Paris is located in the northern part of France, on both banks of the Seine River. It is the capital city of France
and a popular destination for travelers from all over the world.

[All about Paris - This is your essential Paris Guide](https://www.whatparis.com/about-paris.html)
Learn about Paris, the capital of France and one of the most visited tourist destinations in the world. Discover 
its history, culture, cuisine, fashion, art and more in this comprehensive travel guide.

[Where is Paris, France? | Where is Paris Located in The France 
Map](https://www.whereig.com/france/paris-location-map.html)
Paris is the capital and most populous city of France, located on the river Seine in the north-west part of France.
Learn about its geography, climate, history, culture, attractions, and more from this web page.

Out: None

[Step 1: Duration 1.92 seconds| Input tokens: 2,209 | Output tokens: 49]

...

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Le temps à Paris aujourd'hui sera des orages éparpillés avec des pauses occasionnelles sous la pluie. Le haut sera de 30°C et le bas sera de 15°C.


In [24]:
print(manager_agent("Italy weather today"))

╭──────────────────────────────────────────── New run - search_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ Italy country                                                                                                   │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite-001 ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  italy_info = web_search(query="Italy country")                                                                   
  print(italy_info)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Italy - Wikipedia](https://en.wikipedia.org/wiki/Italy)
Italy is a country in Southern and Western Europe with a rich and diverse heritage. Learn about its ancient and 
modern history, its geography, its culture, its role in the European Union and the world, and its achievements in 
various fields.

[Italy | Facts, Geography, History, Flag, Maps, & Population | Britannica](https://www.britannica.com/place/Italy)
Italy, country of south-central Europe, occupying a peninsula that juts deep into the Mediterranean Sea.Italy 
comprises some of the most varied and scenic landscapes on Earth and is often described as a country shaped like a 
boot. At its broad top stand the Alps, which are among the world's most rugged mountains.Italy's highest points are
along Monte Rosa, which peaks in Switzerland, and ...

[Italy | Culture, Facts & Travel - CountryReports](https://www.countryreports.org/country/Italy.htm)
Country Description Italy is a developed democracy with a modern economy. The Holy See is a sovereign entity that 
serves as the ecclesiastical, governmental, and administrative capital of the Roman Catholic Church, physically 
located within the State of the Vatican City inside Rome, with a unique, non-traditional economy. ...

[Italy - The World Factbook](https://www.cia.gov/the-world-factbook/countries/italy/)
Visit the Definitions and Notes page to view a description of each topic.

[Italy Maps & Facts - World Atlas](https://www.worldatlas.com/maps/italy)
Learn about Italy's geography, regions, islands, and volcanoes with maps and facts. Find out where Italy is located
in Europe and the Mediterranean Sea, and what its capital and largest city are.

[Italy - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Italy)
Learn about Italy, a country in Southern Europe with a long history and culture. Find out about its geography, 
people, economy, politics, and more.

[Italy country profile - BBC News](https://www.bbc.com/news/world-europe-17433142)
2022 - Giorgia Meloni becomes country's first female prime minister and leader of Italy's most right-wing led 
government since 1945. Image source, Getty Images Image caption,

[Italy - Country Profile - Nations Online Project](https://www.nationsonline.org/oneworld/italy.htm)
Learn about Italy's history, culture, geography, politics, and more from this comprehensive web page. Find out why 
Italy is famous for its art, food, fashion, and volcanoes.

[Italy | Facts & Information - Infoplease](https://www.infoplease.com/countries/italy)
Infoplease has everything you need to know about Italy. Check out our country profile, full of essential 
information about Italy's geography, history, government, economy, population, culture, religion and languages. If 
that's not enough, click over to our collection of world maps and flags.

[Italy - New World Encyclopedia](https://www.newworldencyclopedia.org/entry/Italy)
Italy (Italia), officially the Italian Republic, is a Southern European country with a population of approximately 
60 million.It comprises the Po River valley, the Italian Peninsula and the two largest islands in the Mediterranean
Sea, Sicily and Sardinia. Called "il Belpaese" (beautiful country) by its inhabitants due to the variety of its 
landscapes and for having the world's largest ...

Out: None

[Step 1: Duration 2.63 seconds| Input tokens: 2,209 | Output tokens: 57]

...

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

Il tempo in Italia oggi dovrebbe essere caldo, con temperature intorno ai 30°C. Ci saranno cieli pieni di sole per la maggior parte della giornata.
